In [23]:
###### Config #####
import sys, os, platform
if os.path.isdir("ds-assets"):
  !cd ds-assets && git pull
else:
  !git clone https://github.com/lutzhamel/ds-assets.git
colab = True if 'google.colab' in os.sys.modules else False
system = platform.system() # "Windows", "Linux", "Darwin"
home = "ds-assets/assets/"
sys.path.append(home)  

Already up to date.


In [24]:
# notebook level imports
import pandas as pd
import numpy as np
import dsutils
np.set_printoptions(formatter={'float_kind':"{:3.2f}".format})
from sklearn import tree
from sklearn import model_selection
from sklearn import metrics

# Model Building and Uncertainty

Building models carries with it a **certain amount of uncertainty**.
Recall that machine learning is an inductive activity: We learn from examples and try to generalize by creating patterns/hypotheses/theories. We use datasets that represent **samples** from much
larger domains in order to learn.  Recall the "black swan problem" where the overall domain of swans contains both white 
and black swans.  But the white swans outnumber the black swans by a substantial margin and therefore, if we are not careful, most samples "D" drawn from
the overall population "X" will only contain white swans as can be seen in the figure below,

<center>
<img 
  src="https://raw.githubusercontent.com/lutzhamel/ds-assets/main/assets/black-swans.png"  
  height="200" 
  width="240">
</center>

This means, if we learn from those samples we will come to the incorrect conclusion that "all swans are white".

What this example illustrates is that the quality of our model is very much dependent on the quality 
of the data samples.  Unfortunately, in most cases the machine learning practitioner has no control over
the construction of the data samples. 
This quality of the sample representation of the domain is a constant source of uncertainty when building models.  We can actually observe this uncertainty even in our simple iris dataset.

In [25]:
df = pd.read_csv(home+"iris.csv")
X  = df.drop(['id','Species'],axis=1)
y = df['Species']

Using train-test splits to build models and reporting the testing accuracy.  We do this five times randomly splitting the iris data into train and test folds.

In [26]:
for i in range(5):
   model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2)
   (X_train, X_test, y_train, y_test) = \
      model_selection.train_test_split(X, 
                                       y, 
                                       train_size=0.7, 
                                       test_size=0.3)
   model.fit(X_train, y_train)
   y_test_model = model.predict(X_test)
   print("Accuracy {}: {:3.2f}"
         .format(i,metrics.accuracy_score(y_test, y_test_model)))

Accuracy 0: 0.91
Accuracy 1: 0.93
Accuracy 2: 0.96
Accuracy 3: 0.96
Accuracy 4: 0.91


Notice the impact the random splits have on the testing accuracy.  But we know this already.  What has changed is our perspective with respect to the source of this variation: **Each split can be seen as randomly sampling a train and a test set from the original domain of all iris flowers**. As we can see, some samples give rise to good models and some samples not so much.  Here we are directly observing the effects of the uncertainty due to the data samples.


Interestingly enough we can observe this uncertainty also in the cross-validation approach although notice that the extreme highs and the extreme lows are no longer present.

In [27]:
for i in range(5):
   model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2)
   create_folds = model_selection.KFold(n_splits=5, shuffle=True) 
   mean_score = model_selection.cross_val_score(model, 
                                                X, 
                                                y, 
                                                cv=create_folds)\
                               .mean()
   print("Mean Accuracy {}: {:3.2f}".format(i, mean_score))

Mean Accuracy 0: 0.95
Mean Accuracy 1: 0.93
Mean Accuracy 2: 0.92
Mean Accuracy 3: 0.95
Mean Accuracy 4: 0.94


As we saw above, this uncertainty reflects into our models. If our data is a poor representation of the domain then the models we construct using it will generalize poorly. If our  data is a good representation of the domain then we can expect that our model will generalize well.


# Classification Confidence Intervals

We use **confidence intervals** in order to quantify the uncertainty discussed above in our model evaluations.

First, let us define confidence intervals formally. Given a model accuracy, *acc*, then the confidence interval is defined as the probability *p* that our model accuracy *acc* lies between some lower bound *lb* and some upper bound *ub*,

$$
Pr(lb \le acc \le ub) = p.
$$

Paraphrasing this equation with *p = 95%*:

> We are 95% percent sure that our model accuracy is not worse than *lb* and not better than *ub*.


Ultimitely we are interested in the lower and upper bounds of the 95% confidence interval.  We can use the following formulas to compute the bounds:

$$ub = acc + 1.96 \sqrt \frac{acc (1 - acc)}{n}$$

$$lb = acc - 1.96 \sqrt \frac{acc (1 - acc)}{n}$$

Here, *n* is the number of observations in the testing dataset used to estimate *acc*. The constant 1.96 is called the *z-score* and expresses the fact that we are computing the 95% confidence interval.

Notice that as we let $n \rightarrow \infty$ both the upper bound and the lower bound tend towards the accuracy.  That is, as we test the model on more and more testing points we become more and more confident that the given accuracy this the correct accuracy.


Let's do a simple example using the function `classification_confint`.

In [28]:
# assume the following
observations = 100
acc = .88

# then
lb,ub = dsutils.classification_confint(acc,observations)
print('Accuracy with confidence interval: {} ({:3.2f},{:3.2f})'.format(acc,lb, ub))

Accuracy with confidence interval: 0.88 (0.82,0.94)


Now, let's do an actual example using our iris dataset.  We want to print out the testing accuracy together with it's 95% confidence interval. We construct a reasonable model and test it

In [29]:
model = tree.DecisionTreeClassifier(criterion='entropy', 
                                    max_depth=3)\
            .fit(X, y)

In [30]:
# compute the accuracy of classifier with confidence interval
acc = model.score(X, y)
observations = X.shape[0]
lb,ub = dsutils.classification_confint(acc,observations)

# print accuracy
print("Accuracy with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})"\
      .format(acc, lb, ub))

Accuracy with confidence interval: 0.97 (0.95, 1.00)


# Regression Confidence Intervals

When performing regression we use the $R^2$ score to examine the quality of our models.  Given that we only use a small training dataset for fitting the model compared to the rest of the data universe it is only natural to ask what the 95% confidence interval for this score might be.  We have a formula for that -- it is not as straight forward as the confidence interval for classification,

$$lb = R^2 - 2\sqrt{\frac{4R^{2}(1-R^{2})^{2}(n-k-1)^{2}}{(n^2 - 1)(n+3)}}$$

$$ub = R^2 + 2\sqrt{\frac{4R^{2}(1-R^{2})^{2}(n-k-1)^{2}}{(n^2 - 1)(n+3)}}$$

Here, *n* is the number of observations in the validation/testing dataset and *k* is the number of independent variables.

In [31]:
# assume the following
rs_score = .75
observations = 100
variables = 4 # independent variables

# then
lb,ub = dsutils.regression_confint(rs_score, observations, variables)
print("R^2 Score with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})"\
      .format(rs_score,lb,ub))

R^2 Score with confidence interval: 0.75 (0.67, 0.83)


Let's look at an actual regression problem and compute the $R^2$ score and it's 95% confidence interval. We will use the cars problem from before.

In [32]:
# get our dataset
cars_df = pd.read_csv(home+"cars.csv")

In [33]:
# build a reasonable model 
model = tree.DecisionTreeRegressor(max_depth=3)\
            .fit(cars_df[['speed']],cars_df[['dist']])

In [34]:
# compute the R^2 score
rs_score = model.score(cars_df[['speed']],cars_df[['dist']])
observations = cars_df.shape[0]
variables = 1
lb,ub = dsutils.regression_confint(rs_score, observations, variables)

# print out R^2 score with its 95% confidence interval
print("R^2 Score with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})"\
      .format(rs_score,lb,ub))

R^2 Score with confidence interval: 0.76 (0.66, 0.87)


# Statistical Significance

Besides giving us an idea of the uncertainty of our model the 95% confidence intervals also have something to say about the significance of scores of different models.  That is, if the confidence intervals overlap then the difference in model performance of two different models on the same dataset is **not statistically significant**.

Consider the following,

In [35]:
observations = 100

# assume first classifier
acc1 = .88  # accuracy of first classifier
lb1,ub1 = dsutils.classification_confint(acc1,observations)
print('Accuracy: {} ({:3.2f},{:3.2f})'.format(acc1,lb1, ub1))

# assume second classifier
acc2 = .92  # accuracy of second classifier
lb2,ub2 = dsutils.classification_confint(acc2,observations)
print('Accuracy: {} ({:3.2f},{:3.2f})'.format(acc2,lb2, ub2))

Accuracy: 0.88 (0.82,0.94)
Accuracy: 0.92 (0.87,0.97)


Even though the second classifier has a better raw accuracy when we look at the confidence intervals of the two classifiers we see that they overlap.  Here we see that the first classifier could potentially have an accuracy of .94 (even better than the raw accuracy of the second classifier).  Furthermore, the confidence interval of the second classifier tells us that that classifier could potentially have an accuracy of .87 which is worse than the raw accuracy of the first classifier.  For this reason we say that the difference in accuracy of two classifiers is not statistically significant if their confidence intervals overlap.

## A Worked Example

Here we use a real-world dataset that tries to predict the sex of abalone given a set of parameters.
First we will construct the optimal model and then we construct a tree with minimal complexity for the same data set and compare the performances using statistical significance.

The optimal tree first.

In [36]:
# get the abalone data
df = pd.read_csv(home+"abalone.csv")

Some basic descriptive statistics:

In [37]:
print(df.shape)
df.head()

(4177, 9)


,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [38]:
df[['sex']].value_counts()

sex
M      1528
I      1342
F      1307
Name: count, dtype: int64

Build our models.

In [39]:
# construct our data matrices
X  = df.drop(columns=['sex'])
y = df[['sex']]

We construct our optimal tree first.

In [40]:
# optimal tree
# setting up grid search
model = tree.DecisionTreeClassifier(random_state=1)
param_grid = {
    'max_depth': list(range(1,11)), # search 1..10
    'criterion': ['entropy', 'gini', 'log_loss']
    }
grid = model_selection.GridSearchCV(model, param_grid, cv=5)

# performing grid search
grid.fit(X,y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=1),
             param_grid={'criterion': ['entropy', 'gini', 'log_loss'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

In [41]:
# compute the accuracy of optimal classifier with conf interval
acc = grid.best_estimator_.score(X, y)
observations = X.shape[0]
lb,ub = dsutils.classification_confint(acc,observations)

# print accuracy
print("Accuracy of optimal classifier: {:3.2f} ({:3.2f}, {:3.2f})"
      .format(acc, lb, ub))

Accuracy of optimal classifier: 0.59 (0.57, 0.60)


Now we construct the minimal tree with max depth of 2.  We chose two because at minimum we need two nested if-then-else statements in order to distinguish three different labels.

In [42]:
# minimal complexity tree: depth 2
# create our model object
model = tree.DecisionTreeClassifier(criterion='entropy', 
                                    max_depth=2, 
                                    random_state=1)

# train
model.fit(X,y)

DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=1)

In [43]:
# compute the accuracy of minimal classifier
acc = model.score(X, y)
observations = X.shape[0]
lb,ub = dsutils.classification_confint(acc,observations)

# print accuracy
print("Accuracy of minimal classifier: {:3.2f} ({:3.2f}, {:3.2f})"
      .format(acc, lb, ub))

Accuracy of minimal classifier: 0.53 (0.52, 0.55)


**Observation**: The confidence intervals are not overlapping, therefore **the performance difference is statistically significant**! That means the optimal model indeed performs better than the minimal tree.

# Project

Please see BrightSpace for project #3

# Midterm

The midterm will cover everything up to and including the material in project #3